In [1]:
import logging
import tensorflow as tf
import glob
import os
import cPickle as pickle
from librosa import feature as ftr
import soundfile as sf
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import librosa.feature as lfr
import matplotlib.pyplot as plt
# from keras.utils.visualize_util import plot

from keras.metrics import categorical_accuracy, binary_accuracy
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D
from keras.datasets import mnist

%matplotlib inline

/Users/zhouqiang/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/zhouqiang/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
Using TensorFlow backend.


In [2]:
pf=open('/Users/zhouqiang/Downloads/urbansound/UrbanSound/src/mfcc.pkl', 'r')

starts at 9:20

In [3]:
def get_file_num(fn):
    return fn.split('/')[-1].split('.')[0]

def get_labels():
    parent_dir="/Users/zhouqiang/Downloads/urbansound/UrbanSound/data"
    sub_dirs=['air_conditioner','children_playing','drilling','gun_shot','siren', 'car_horn','dog_bark','engine_idling','jackhammer','street_music']
    labels = {'air_conditioner':1,'children_playing':2,'drilling':3,'gun_shot':4,'siren':5,
             'car_horn':6,'dog_bark':7,'engine_idling':8,'jackhammer':9,'street_music':10
            }

    wav_file_ext, mp3_file_ext = "*.wav", "*.mp3"
    labels_dict = dict()

    result = []
    for sub_dir in sub_dirs:
        label = labels[sub_dir]
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, wav_file_ext)):
            fn = get_file_num(fn)
            #print "hello"
            labels_dict[fn]=label
#            result.append( (fn, label) )

    for sub_dir in sub_dirs:
        label = labels[sub_dir]
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, mp3_file_ext)):
            fn = get_file_num(fn)
            #print "world"
            labels_dict[fn]=label
#            result.append( (fn, label) )
    return labels_dict

In [4]:
labels_dict = get_labels()

In [5]:
labels=[]
mfccs=[]
lengths= []
mfcc_count = 0
try:
    #for i in range(100):
    for i in range(1106):
    #for i in range(2):
        #print i
        mfcc_count += 1
        tup=pickle.load(pf)
        #print tup
        label = labels_dict[tup[0]]
        
        labels.append(label)
        lengths.append(len(tup[1][0]))
        mfccs.append( tup[1] )
except EOFError:
    print mfcc_count

In [6]:
len(labels), len(mfccs), mfccs[0].shape

(1106, 1106, (20, 7930))

In [ ]:
plt.imshow(mfccs[0][:,:200])

In [7]:
data, class_labels  = [], []
for mfcc, m_label in zip(mfccs, labels):
    # slice (20, x) into many of these (20, 35) ...... 
    # throw away the last vector
    mfcc_len = mfcc.shape[1]
    step_size = 35
    num_seg = mfcc_len/step_size
    for i in range(num_seg):
        start=i*step_size
        end=start+step_size
        data.append(mfcc[:,start:end])
        class_labels.append(m_label)

In [13]:
len(data), len(class_labels)

(148023, 148023)

In [8]:
# ONLY TAKE THE FIRST 400 DATA POINTS

class_data = []
for img in data:
    class_data.append( np.array(img).flatten() )

# for t in range(400):
#     class_data.append( np.array(img).flatten() )

In [9]:
model = Sequential()
d1 = Dense(32, input_dim=700)
model.add(d1)
model.add(Activation('relu'))

d2 = Dense(10, input_dim=400)
model.add(d2)
model.add(Activation('softmax'))
# plot(model)

In [10]:
training_data_X=np.array(class_data)
print training_data_X.shape

# only take the first 400 points
# training_Y = np.array(labels[:400]).reshape((-1,1))

label_vec = { 1: [1,0,0,0,0,0,0,0,0,0], 2: [0,1,0,0,0,0,0,0,0,0] , 3: [0,0,1,0,0,0,0,0,0,0], 4: [0,0,0,1,0,0,0,0,0,0]
            , 5: [0,0,0,0,1,0,0,0,0,0], 6: [0,0,0,0,0,1,0,0,0,0], 7: [0,0,0,0,0,0,1,0,0,0], 8: [0,0,0,0,0,0,0,1,0,0]
            , 9: [0,0,0,0,0,0,0,0,1,0], 10:[0,0,0,0,0,0,0,0,0,1]
            }
tmp_training_Y = []
# for l in labels[:400]:
for l in class_labels:
    tmp_training_Y.append( label_vec[l] )
training_Y = np.array( tmp_training_Y )
print training_Y.shape

(148023, 700)
(148023, 10)


In [12]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit( training_data_X, training_Y, nb_epoch=200, batch_size=32)

#model.fit(class_data, class_labels, nb_epoch=10, batch_size=32)
# model.fit(X_train[:100], y_train[:100], nb_epoch=10, batch_size=32)

Epoch 1/200
148023/148023 [==============================] - 8s - loss: 11.8808 - acc: 0.2629     
Epoch 2/200
148023/148023 [==============================] - 8s - loss: 11.8808 - acc: 0.2629     
Epoch 3/200
148023/148023 [==============================] - 8s - loss: 11.8808 - acc: 0.2629     
Epoch 4/200
148023/148023 [==============================] - 8s - loss: 11.8808 - acc: 0.2629     
Epoch 5/200
148023/148023 [==============================] - 7s - loss: 11.8808 - acc: 0.2629     
Epoch 6/200
148023/148023 [==============================] - 8s - loss: 11.8808 - acc: 0.2629     
Epoch 7/200
148023/148023 [==============================] - 8s - loss: 11.8808 - acc: 0.2629     
Epoch 8/200
148023/148023 [==============================] - 9s - loss: 11.8808 - acc: 0.2629     
Epoch 9/200
148023/148023 [==============================] - 8s - loss: 11.8808 - acc: 0.2629     
Epoch 10/200
148023/148023 [==============================] - 8s - loss: 11.8808 - acc: 0.2629     
Epoch 11/

In [ ]:
y_pred = model.predict( training_data_X )

In [ ]:
y_pred.shape, training_Y.shape

In [ ]:
def my_accuracy(y_true, y_pred):
    count, total = 0, y_true.shape[0]
    y_true = y_true / 1.0
    for t, p in zip(y_true, y_pred):
#         print t, p
#         break
        if (t==p).all():
            count += 1
    return count/float(total)

In [ ]:
def pred_to_vec(y_pred):
    # y_pred is a 10 element vector
    # take the highest score index, then generate a vector with 1 at that position
    return [ np.argmax(p) for p in y_pred ]


In [ ]:
tmp=pred_to_vec(y_pred)

In [ ]:
tmp_df=pd.Series(tmp)

In [ ]:
tmp_df.value_counts()

In [ ]:
# categorical_accuracy(training_Y, y_pred)
print my_accuracy(training_Y, y_pred)

In [ ]:
a=np.array([1,0,0])
b=np.array([1,0,1])
print (a==b).all()


n_dim = 120
n_classes = 10
n_hidden_units_one = 280 
n_hidden_units_two = 300
sd = 1 / np.sqrt(n_dim)
learning_rate = 0.01

X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,n_classes])

W_1 = tf.Variable(tf.random_normal([n_dim,n_hidden_units_one], mean = 0, stddev=sd))
b_1 = tf.Variable(tf.random_normal([n_hidden_units_one], mean = 0, stddev=sd))
h_1 = tf.nn.tanh(tf.matmul(X,W_1) + b_1)


W_2 = tf.Variable(tf.random_normal([n_hidden_units_one,n_hidden_units_two], mean = 0, stddev=sd))
b_2 = tf.Variable(tf.random_normal([n_hidden_units_two], mean = 0, stddev=sd))
h_2 = tf.nn.sigmoid(tf.matmul(h_1,W_2) + b_2)


W = tf.Variable(tf.random_normal([n_hidden_units_two,n_classes], mean = 0, stddev=sd))
b = tf.Variable(tf.random_normal([n_classes], mean = 0, stddev=sd))
y_ = tf.nn.softmax(tf.matmul(h_2,W) + b)

init = tf.initialize_all_variables()

cost_function = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(y_), reduction_indices=[1])) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sd

print tf.random_normal( [10], mean=0, stddev=sd)

mm=tf.matmul(h_2,W)+b

mm.get_shape()

b_val = b.value()

type(accuracy)